# Quantitative Momentum Screener

## Table of Contents

## Project Overview

Momentum investing refers to investing in assets that have increased in price the most.

This project aims to build a momentum trading strategy.